<a href="https://colab.research.google.com/github/ctk03272/deeplearningstudy/blob/main/chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chapter5
- 수치 미분은 단순하고 구현도 쉽지만 계산이 오래 걸린다는 게 단점이다. 가중치 매개변수의 기울기를 효율적으로 계산하는 '오차역전파법'을 배워 본다.
- 오차역전파법은 수식과 그래프 두가지로 이해할 수 있다.

## 5.1 계산 그래프
- 계산과정을 그래프로 나타낸 것

### 5.1.1 계산 그래프로 풀다.
- 계산을 왼쪽에서 오른쪽으로 진행하는 단계를 순전파
- 오른쪽에서 왼쪽으로 전파가 역전파

### 5.1.2 국소적 계산
- 계산그래프의 특징은 국소적 계산을 전파함으로 최종 결과를 얻는다는 것이다.
- 전체 계산이 아무리 복잡해져도 각 단계에서 하는 일은 해당 노드의 국소적 계산이다.

### 5.1.3 왜 계산 그래프로 푸는가?
- 실제 계산그래프를 사용하는 가장 큰 이유는 역전파를 통해 미분을 효율적으로 계산 할 수 있다
- 계산 그래프의 이점은 순전파와 역전파를 활용해서 각 변수의 미분을 효율적으로 구할 수 있다.

## 5.2. 연쇄법칙
- 국소적 미분을 전달하는 원리는 연쇄법칙에 따른 것이다.

### 5.2.1 계산 그래프의 역전파
- 신호의 노드에 국소적 미분을 곱한 후 다음 노드로 전달하는 것이다.

### 5.3.1 덧셈 노드의 역전파
- 덧셈 노드의 역전파는 입력 값을 그대로 흘려보낸다.

### 5.3.2 곱셈 노드의 역전파
- 곱셈 노드 역전파는 상류의 값에 순전파 떄의 입력 신호들을 서로 바꾼 값을 곱해서 하류로 보낸다.
- 따라서 곱셈의 역전파는 순방향 입력 신호의 값이 필요하다. 그래서 곱셉 노드를 구현할 떄는 순전파의 입력 신호를 변수에 저장해 둔다.

## 5.4 단순한 계층 구현하기

### 5.4.1 곱셈 계층
- 모든 계층은 forward()와 backward()라는 공통 메서드를 갖도록 한다.


In [ ]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy


class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

In [ ]:



apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

price: 220
dApple: 2.2
dApple_num: 110
dTax: 200


In [ ]:


apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


# 5.5 활성화 함수 계층 구현하기

## 5.5.1 ReLU 계층
- 0보다 클때는 선형 함수 0보다 작을떄는 0인 함수이다
- ReLU 계층은 전기회로의 스위치에 비유할 수 있다.

In [ ]:

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

### 5.5.2 Sigmoid 계층
- 시그모이드 함수를 사용한다.

In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## 5.6 Affine/Softmax 계측 구현하기

### 5.6.1 Affine 계층
- 신경파의 순전파 때 수행하는 행렬의 곱은 기하학에서는 어파인변환 이라고 한다. 그래서 이 책에서는 어파인 변환을 수행하는 처리를 Affine 계층 이라는 이름으로 구현한다.

In [ ]:

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b

        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        dx = dx.reshape(*self.original_x_shape)
        return dx

### 5.6.3 Softmax-with-Loss 계층
- 소프트맥스 함수는 입력 값을 정규화하여 출력
- 신경망에서 수행하는 작업은 학습과 추론 두가지이다. 앞장에서 본거같이 추론시에는 일반적으로 SoftMax 계층을 사용하지 않는다.

In [ ]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size:
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size

        return dx

# 5.7 오차역전파법 *구현하기*

In [24]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [25]:
# coding: utf-8
import sys, os


import numpy as np
from mnist import load_mnist
from layers import *
from gradient import numerical_gradient

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.10911666666666667 0.1119
0.9057166666666666 0.9093
0.91795 0.9206
0.9308166666666666 0.9325
0.94205 0.942
0.95065 0.9498
0.9561666666666667 0.9537
0.9613166666666667 0.9584
0.9642166666666667 0.9595
0.9670166666666666 0.9627
0.9684333333333334 0.9625
0.9705166666666667 0.9639
0.97195 0.9664
0.9747166666666667 0.9664
0.9763333333333334 0.9681
0.9774833333333334 0.9704
0.978 0.9707


In [28]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:4.277259140813999e-10
b1:2.342659291295181e-09
W2:5.713298156757819e-09
b2:1.3990672419750271e-07


In [30]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)

import numpy as np
from mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.14445 0.1464
0.9036666666666666 0.9079
0.9239166666666667 0.9273
0.9366666666666666 0.9389
0.9468333333333333 0.9465
0.95185 0.9513
0.95575 0.9531
0.9594666666666667 0.9563
0.9635833333333333 0.9588
0.9670333333333333 0.9617
0.9664833333333334 0.9621
0.9711666666666666 0.9637
0.97325 0.9648
0.9752833333333333 0.9674
0.9748 0.9661
0.9765166666666667 0.9685
0.97865 0.97
